In [1]:
import os                              
import numpy as np
from numpy import number                    
import pandas as pd                     
import matplotlib.pyplot as plt  
import math
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term
from azure.quantum.optimization import ParallelTempering
from azure.quantum.optimization import Term, SlcTerm

<h3 style="color:blue;"> Data Preprocessing </h3>

In [2]:
stocks = [stock.split('.')[0] for stock in sorted(os.listdir("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)"))]
stocks = stocks[1:]

print(stocks)

['ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADANITRANS', 'ASIANPAINT', 'ATGL', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BHARTIARTL', 'BRITANNIA', 'COALINDIA', 'DABUR', 'DIVISLAB', 'DMART', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HINDUNILVR', 'HINDZINC', 'ICICIBANK', 'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PIDILITIND', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO']


In [3]:
dates = pd.date_range('2020-06-27', '2022-06-27') 
data = pd.DataFrame({'Time': dates})


In [4]:
for stock in stocks:
    prices = pd.read_csv("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)/"+stock+".csv", usecols=['Date', 'WAP'])
    #print(prices)
    prices['Date'] = pd.to_datetime(prices['Date'], dayfirst = True)  
    #print(prices)
    prices.rename(                                                            
        columns={"Date": "Time", "WAP": stock},
        inplace=True
    )
    data = pd.merge(data,prices)


In [5]:

cp = data .drop(['Time'], axis=1).tail(1).to_numpy()
cp = cp[0]

In [6]:
# Calculating daily returns for each stock
r = data[(data['Time'] >= '2020-06-27')] \
    .drop(['Time'], axis=1) \
    .pct_change(fill_method='ffill')
r.head()

,ADANIENT,ADANIGREEN,ADANIPORTS,ADANIPOWER,ADANITRANS,ASIANPAINT,ATGL,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,...,SBIN,SIEMENS,SUNPHARMA,TATAMOTORS,TATASTEEL,TCS,TECHM,TITAN,ULTRACEMCO,WIPRO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.003303,-0.049927,0.014850,-0.004988,-0.037398,0.005966,-0.015178,0.006920,0.005066,0.011924,...,0.000865,0.007351,-0.017048,0.000601,0.047004,-0.003164,-0.000443,0.010112,0.021308,-0.007733
2,-0.010636,-0.049903,-0.006293,-0.009519,0.007948,-0.006665,-0.002781,0.035394,-0.002453,0.016905,...,0.014059,-0.015242,-0.008919,0.004799,-0.025907,0.001420,-0.005275,-0.004628,0.004782,-0.002084
3,0.007015,-0.000395,0.007715,0.002276,0.010551,-0.002579,0.047833,0.009497,0.007152,0.033348,...,0.018608,0.004666,0.005206,0.016795,0.021360,0.020015,0.022206,0.021991,0.004248,0.021089
4,0.017205,0.102695,0.041561,-0.001960,-0.013318,0.011123,-0.009999,-0.001076,0.021797,-0.001638,...,-0.006828,0.035929,0.009218,0.029517,0.005488,0.021521,0.022392,0.023023,0.000064,0.006637


In [7]:
# Calculating mean and covariance matrix for each stocks
mu = r.mean().to_numpy()
sigma = r.cov().to_numpy()
n=len(stocks)

In [8]:
'''
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "e87e6b08-4913-4522-9206-ba18a56ee0fa",
    resource_group = "AzureQuantum",
    name = "MTCProjectQuantumSharique",
    location = "East US"
)


from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "6e02aef9-2670-4418-92b8-7659cf5605d2",
    resource_group = "azurequantum",
    name = "workspace1",
    location = "japaneast"
)
'''

from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "ad18d2e0-288d-409a-bc11-5231714f4fb8",
    resource_group = "rg-mtc-poc-quantum",
    name = "ws-mtc-poc-quantum",
    location = "japaneast"
)



<h3 style="color:blue;">Optimized Portfolio </h3>

In [9]:
import functions


In [10]:

t1= functions.return_terms(mu,n)
t2= functions.risk(sigma, n, 1)
terms=[]
terms=t1+t2
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)
solver = ParallelTempering(workspace)
solution = solver.optimize(problem)


Number of terms in return:  49
Number of terms in risk:  2401
......

In [11]:
print(solution['configuration'])
result = solution['configuration']
#print(type(result))

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()
#print(daily_prices)

for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(cp[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", prices_selected)

{'0': 1, '1': 1, '2': 0, '3': 1, '4': 1, '5': 0, '6': 1, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 1, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 1, '25': 0, '26': 0, '27': 1, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 1, '40': 1, '41': 1, '42': 1, '43': 0, '44': 0, '45': 0, '46': 1, '47': 0, '48': 0}

Selected stocks are -
 ['ADANIENT', 'ADANIGREEN', 'ADANIPOWER', 'ADANITRANS', 'ATGL', 'HCLTECH', 'INFY', 'JSWSTEEL', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TITAN']

Price of selected stocks are -
 [2188.648926689398, 1906.6661531149095, 273.4378450035677, 2149.2141705744434, 2295.5344228094573, 997.2557371818474, 1476.0546642603206, 581.4109040683994, 461.347105474746, 2374.566375643609, 842.2730643765593, 415.93760931737654, 2046.4069499762488]


In [14]:
Budget = 1000000
risk_aversion_index = 1
daily_returns = functions.find_expected_daily_return(mu,selected_stocks,index_prices_selected)
annual_returns_percent = round((daily_returns*252 *100),3)
annual_returns = round(Budget+ (daily_returns*252*Budget),1)
risk_percent = round(functions.find_risk(selected_stocks,n,result,sigma)*(252**0.5),4)

In [16]:
# Printing results
print("OPTIMIZSED PORTFOLIO :\n")
no_of_shares = functions.distributed_budget(1000000, arr= selected_stocks, cp=cp, index_prices_selected= index_prices_selected)
print("Expected Annual Return on Investment : ", annual_returns_percent, "%")
print("\nConstraints:")
print("\nBudget : ",  u"\u20B9", Budget)
print("Risk Aversion index of :" , risk_aversion_index,"\n")
print("Expected Annual Return: ",  u"\u20B9", annual_returns)
print("Annual Risk percentage: ", risk_percent ,"%")

OPTIMIZSED PORTFOLIO :

ADANIENT  :  35  shares
ADANIGREEN  :  40  shares
ADANIPOWER  :  281  shares
ADANITRANS  :  35  shares
ATGL  :  33  shares
HCLTECH  :  77  shares
INFY  :  52  shares
JSWSTEEL  :  132  shares
SBIN  :  166  shares
SIEMENS  :  32  shares
SUNPHARMA  :  91  shares
TATAMOTORS  :  184  shares
TITAN  :  37  shares
Expected Annual Return on Investment :  77.261 %

Constraints:

Budget :  ₹ 1000000
Risk Aversion index of : 1 

Expected Annual Return:  ₹ 1772609.5
Annual Risk percentage:  22.0944 %


<h3 style="color:blue;"> Diversified Portfolio </h3>
Selecting a portfolio after applying a budget constraint which restricts number
of stocks.

In [17]:
terms=[]
t1 = functions.return_terms(mu,n)
t2 = functions.risk(sigma,n,1)
t3 = functions.budget_constraint(2*n,n,15)
terms= t1+ t2 + t3
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)
solver = ParallelTempering(workspace)
solution = solver.optimize(problem)

Number of terms in return:  49
Number of terms in risk:  2401
........

In [18]:
print(solution['configuration'])
result = solution['configuration']

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()


for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(cp[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", index_prices_selected)

{'0': 1, '1': 1, '2': 0, '3': 1, '4': 1, '5': 0, '6': 1, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 1, '24': 1, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 1, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 1, '39': 1, '40': 1, '41': 1, '42': 1, '43': 0, '44': 1, '45': 1, '46': 0, '47': 0, '48': 0}

Selected stocks are -
 ['ADANIENT', 'ADANIGREEN', 'ADANIPOWER', 'ADANITRANS', 'ATGL', 'ICICIBANK', 'INFY', 'M&M', 'SBILIFE', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TCS', 'TECHM']

Price of selected stocks are -
 [0, 1, 3, 4, 6, 23, 24, 30, 38, 39, 40, 41, 42, 44, 45]


In [21]:
Budget = 1000000
risk_aversion_index = 1
stock_appetite = 15
daily_returns = functions.find_expected_daily_return(mu,selected_stocks,index_prices_selected)
annual_returns_percent = round((daily_returns*252 *100),3)
annual_returns = round(Budget+ (daily_returns*252*Budget),1)
risk_percent = round(functions.find_risk(selected_stocks,n,result,sigma)*(252**0.5),4)


In [22]:
# Printing results

print("\n DIVERSIFIED PORTFOLIO:")
no_of_shares = functions.distributed_budget(1000000, selected_stocks, cp, index_prices_selected)

print("\nConstraints:")
print("\nBudget : ",  u"\u20B9", Budget)
print("Stock Appetite: ", stock_appetite , " stocks")
print("Risk Aversion index of :" , 1,"\n")
print("Expected Annual Return on Investment : ", annual_returns_percent, "%")
print("Expected Annual Return: ",  u"\u20B9", annual_returns)
print("Annual Risk percentage: ", risk_percent ,"%")



 DIVERSIFIED PORTFOLIO:
ADANIENT  :  30  shares
ADANIGREEN  :  34  shares
ADANIPOWER  :  243  shares
ADANITRANS  :  31  shares
ATGL  :  29  shares
ICICIBANK  :  92  shares
INFY  :  45  shares
M&M  :  61  shares
SBILIFE  :  61  shares
SBIN  :  144  shares
SIEMENS  :  28  shares
SUNPHARMA  :  79  shares
TATAMOTORS  :  160  shares
TCS  :  20  shares
TECHM  :  65  shares

Constraints:

Budget :  ₹ 1000000
Stock Appetite:  15  stocks
Risk Aversion index of : 1 

Expected Annual Return on Investment :  68.773 %
Expected Annual Return:  ₹ 1687729.5
Annual Risk percentage:  20.5419 %
